## RNN

1. RNN  
|--------------------------|  
입력 데이터에 정답이 있음  
초기 일정 구간을 정답으로 사용  
1-10번째 입력데이터로 예측한 11번째 => 2-11번째 입력데이터로 예측한 12번째 ...  
연속적으로 데이터 활용  
중복된 데이터셋으로 구성  
이전에 예측된 결과도 입력데이터로 사용  
예측문제에 사용  
주 센서를 예측하는데 보조 센서 여러개가 입력데이터로만 사용 가능. 주  센서는 본인 입력데이터와 보조센서 입력데이터를 이용하여 예측.  
입력값을 수치형으로 변환해야함 !!  
입력데이터만큼 히든레이어 수치 지정 !!  
ex. 주식시세 예측, 센서데이터로 알아보는 장비 예지보전  

---
2. DNN  
|--||--||--||--||--||--||--|  
입력 데이터와 무관한 정답을 사용  
고양이 이미지  
분류문제에 사용  


In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = "2"   # 경고 메시지 제거

import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

In [2]:
tf.random.set_seed(22)
np.random.seed(22)
assert tf.__version__.startswith('2.')

batch_size = 128
total_words = 10000
max_review_len = 80
embedding_len = 100

In [3]:
# 영화관람평을 수치로 변환 후 긍정인지 부정인지 분류
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.imdb.load_data(num_words=total_words)
    
x_train  = tf.keras.preprocessing.sequence.pad_sequences(x_train, maxlen=max_review_len)  # padding 작업 (80자로 맞추기)
x_test  = tf.keras.preprocessing.sequence.pad_sequences(x_test, maxlen=max_review_len)    # 길이가 길면 max_review_len만큼 자르고
                                                                                          # 길이가 짧으면 그만큼 0을 채워줌.

<__array_function__ internals>:180: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
C:\Users\user13\anaconda3\envs\tf2_book\lib\site-packages\tensorflow\python\keras\datasets\imdb.py:159: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
C:\Users\user13\anaconda3\envs\tf2_book\lib\site-packages\tensorflow\python\keras\datasets\imdb.py:160: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or s

In [4]:
print('x : ',x_train[0])
print('y : ',y_train[0])

x :  [  15  256    4    2    7 3766    5  723   36   71   43  530  476   26
  400  317   46    7    4    2 1029   13  104   88    4  381   15  297
   98   32 2071   56   26  141    6  194 7486   18    4  226   22   21
  134  476   26  480    5  144   30 5535   18   51   36   28  224   92
   25  104    4  226   65   16   38 1334   88   12   16  283    5   16
 4472  113  103   32   15   16 5345   19  178   32]
y :  1


In [5]:
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.shuffle(10000).batch(batch_size, drop_remainder=True)  # 128개씩 자르다가 마지막 자투리는 버림

test_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_data = train_data.batch(batch_size, drop_remainder=True)

In [6]:
x_train.shape        # 25000, 80
y_train.shape        # 25000, 
x_test.shape         # 25000, 80
y_test.shape         # 25000,

(25000,)

In [11]:
# 신경망 구축
class RNN_Build(tf.keras.Model):
    def __init__(self, units):
        super(RNN_Build, self).__init__()
        self.embedding = tf.keras.layers.Embedding(total_words, embedding_len, input_length=max_review_len)  # 자연어 처리에 필요
        
        self.rnn = tf.keras.Sequential([
            tf.keras.layers.LSTM(units, dropout=0.2, return_sequences=True),
            tf.keras.layers.LSTM(units, dropout=0.2)
        ])
        self.outlayer = tf.keras.layers.Dense(1)
        
    def call(self, inputs, training=None):        # 전체 과정
        x = inputs               # 입력
        x = self.embedding(x)    # embedding 과정
        x = self.rnn(x)          # rnn 과정
        x = self.outlayer(x)     # 출력 레이어
        prob = tf.sigmoid(x)     # 시그모이드
        
        return prob

In [12]:
units = 64
epochs = 4

model = RNN_Build(units)

model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss=tf.losses.BinaryCrossentropy(),
              metrics=['accuracy'],
              experimental_run_tf_function=False)

In [13]:
history = model.fit(train_data, epochs=epochs, validation_data=test_data)

Epoch 1/4


NotImplementedError: in user code:

    C:\Users\user13\anaconda3\envs\tf2_book\lib\site-packages\tensorflow\python\keras\engine\training.py:806 train_function  *
        return step_function(self, iterator)
    C:\Users\user13\AppData\Local\Temp\ipykernel_10944\2778857207.py:16 call  *
        x = self.rnn(x)          # rnn 과정
    C:\Users\user13\anaconda3\envs\tf2_book\lib\site-packages\tensorflow\python\keras\engine\base_layer.py:985 __call__  **
        outputs = call_fn(inputs, *args, **kwargs)
    C:\Users\user13\anaconda3\envs\tf2_book\lib\site-packages\tensorflow\python\keras\engine\sequential.py:386 call
        outputs = layer(inputs, **kwargs)
    C:\Users\user13\anaconda3\envs\tf2_book\lib\site-packages\tensorflow\python\keras\layers\recurrent.py:663 __call__
        return super(RNN, self).__call__(inputs, **kwargs)
    C:\Users\user13\anaconda3\envs\tf2_book\lib\site-packages\tensorflow\python\keras\engine\base_layer.py:985 __call__
        outputs = call_fn(inputs, *args